In [ ]:


# ==========================================
# 1. 準備
# ==========================================
!pip install japanize-matplotlib

import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import japanize_matplotlib
import logging
from datetime import date, timedelta

logging.getLogger('prophet').setLevel(logging.ERROR)
logging.getLogger('cmdstanpy').setLevel(logging.ERROR)

# ==========================================
# 2. 設定（実運用向け自動設定）
# ==========================================
FILE_PATH = '/content/R予測用.CSV'

# 今日の日付を基準に設定
TODAY = date.today()
PREDICTION_DAYS = 30  # 30日先まで
END_DATE = TODAY + timedelta(days=PREDICTION_DAYS)

# 特売対象の商品名
TARGET_ITEM_NAME = 'ＰＨバナナＳＷ２００'

# 保存ファイル名に実行日を付与
today_str = TODAY.strftime('%Y%m%d')
pdf_file_name = f'{today_str}_需要予測結果.pdf'
csv_file_name = f'{today_str}_需要予測結果.csv'

# ==========================================
# 3. 実行処理
# ==========================================
df = pd.read_csv(FILE_PATH, parse_dates=['出荷日'])
df['商品識別ID'] = df['商品コード'].astype(str) + "_" + df['商品名'] + "(" + df['規格名'].fillna('無') + ")"

all_results = []
all_items = df['商品識別ID'].unique()

print(f"予測を開始します: {TODAY} から {PREDICTION_DAYS}日間")

with PdfPages(pdf_file_name) as pdf:
    for i, item_id in enumerate(all_items):
        try:
            target_data = df[df['商品識別ID'] == item_id].groupby('出荷日')[['数量']].sum().reset_index()
            df_prophet = target_data.rename(columns={'出荷日': 'ds', '数量': 'y'})

            # --- 特売フラグの設定 ---
            if TARGET_ITEM_NAME in item_id:
                df_prophet['is_sale_day'] = df_prophet['ds'].dt.day.apply(lambda x: 1 if x in [8, 18, 28] else 0)
                use_regressor = True
            else:
                df_prophet['is_sale_day'] = 0
                use_regressor = False

            # 学習データ：CSVにある全データを使用
            train_df = df_prophet.copy()
            if len(train_df) < 14: continue

            # --- モデル構築 ---
            model = Prophet(yearly_seasonality=True, weekly_seasonality=True, daily_seasonality=False)
            model.add_country_holidays(country_name='JP')

            if use_regressor:
                model.add_regressor('is_sale_day')

            model.fit(train_df)

            # --- 予測作成 (本日より30日分) ---
            last_date = train_df['ds'].max().date()
            days_to_today = (TODAY - last_date).days
            total_periods = max(0, days_to_today) + PREDICTION_DAYS

            future = model.make_future_dataframe(periods=total_periods)
            future['is_sale_day'] = future['ds'].dt.day.apply(lambda x: 1 if x in [8, 18, 28] else 0) if use_regressor else 0

            forecast = model.predict(future)

            # --- グラフ作成 ---
            fig1, ax = plt.subplots(figsize=(10, 5))
            model.plot(forecast, ax=ax)

            # 【修正箇所】実績値（赤点）を大きく、最前面に描画
            ax.scatter(df_prophet['ds'], df_prophet['y'],
                       color='red',
                       s=35,               # 点のサイズを大きく
                       label='実際の実績',
                       edgecolors='white', # 白い縁取りで輪郭をはっきり
                       linewidths=0.5,
                       zorder=5)           # 重なり順を一番上に設定

            # 本日線
            ax.axvline(pd.to_datetime(TODAY), color='green', linestyle='--', label='本日 (予測開始)', zorder=6)

            plt.title(f"【需要予測】\n{item_id}", fontsize=10)
            plt.xlim(pd.to_datetime(TODAY) - pd.Timedelta(days=30), pd.to_datetime(END_DATE) + pd.Timedelta(days=2))
            plt.legend()
            pdf.savefig(fig1, bbox_inches='tight')
            plt.close(fig1)

            # --- CSV用蓄積 (本日以降の予測分のみ) ---
            res = forecast[forecast['ds'] >= pd.to_datetime(TODAY)].copy()
            res['商品識別情報'] = item_id
            all_results.append(res)

        except Exception as e:
            print(f"エラー：{item_id} ({e})")

# ==========================================
# 4. 保存
# ==========================================
if all_results:
    final_df = pd.concat(all_results, ignore_index=True)
    output_cols = ['商品識別情報', '日付', '予測数量', 'yhat_lower', 'yhat_upper']

    final_df.rename(columns={'ds': '日付', 'yhat': '予測数量'}, inplace=True)
    final_df = final_df[[c for c in output_cols if c in final_df.columns]]

    final_df.to_csv(csv_file_name, index=False, encoding='utf-8-sig')
    print(f"完了しました！\nPDF: {pdf_file_name}\nCSV: {csv_file_name}")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 58.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize-matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=67cf2290ee15dc4dfd15588ed38a97bbe00875cd1e6246d517d3bf3aff707bcf
  Stored in directory: /root/.cache/pip/wheels/c1/f7/9b/418f19a7b9340fc16e071e89efc379aca68d40238b258df53d
Successfully built japanize-matplotlib
予測を開始します: 2026-01-13 から 30日間
完了しました！
PDF: 20260113_需要予測結果.pdf
CSV: 20260113_需要予測結果.csv
